# Pre-processing News Data

## Bài toán
Dữ liệu gồm n văn bản phân vào 10 chủ đề khác nhau. Cần biễu diễn mỗi văn bản dưới dạng một vector số thể hiện cho nội dụng của văn bản đó.

## Mục lục
- Load dữ liệu từ thư mục
- Loại bỏ các stop words
- Sử dụng thư viện để mã hóa TF-IDF cho mỗi văn bản

## Phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

In [8]:
!pip install pyvi

In [9]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_files
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

%matplotlib inline

## Load dữ liệu từ thư mục

Cấu trúc thư mục như sau

- data/news_vnexpress/

    - Kinh tế:
        - bài báo 1.txt
        - bài báo 2.txt
    - Pháp luật
        - bài báo 3.txt
        - bài báo 4.txt

In [10]:
INPUT = '/content/drive/MyDrive/Dataset/news_vnexpress'
os.makedirs("images",exist_ok=True)  # thư mục lưu các các hình ảnh trong quá trình huấn luyện và đánh gía

In [11]:
# Statistic
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')

n = 0
for label in os.listdir(INPUT):
    label_path = os.path.join(INPUT, label)
    # Check if the item is a directory
    if os.path.isdir(label_path):
        num_files = len(os.listdir(label_path))
        print(f'{label}: {num_files}')
        n += num_files

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
du-lich: 54
suc-khoe: 25
kinh-doanh: 262
khoa-hoc: 144
phap-luat: 59
giai-tri: 201
thoi-su: 0
doi-song: 120
the-thao: 0
giao-duc: 105
-------------------------
Tổng số văn bản: 970


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# load data
data_train = load_files(container_path=INPUT, encoding="utf-8")
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:1])
# print(data_train.data[0:1])
print(data_train.target[0:1])
print(data_train.data[0:1])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
doi-song - 0
du-lich - 1
giai-tri - 2
giao-duc - 3
khoa-hoc - 4
kinh-doanh - 5
phap-luat - 6
suc-khoe - 7
the-thao - 8
thoi-su - 9
--------------------------
['/content/drive/MyDrive/Dataset/news_vnexpress/phap-luat/00047.txt']
[6]
['Giao dịch chỉ viết giấy tay có hàng xóm làm chứng. Tôi sử dụng nhà đất ổn định từ khi mua đến nay. Xin hỏi, tôi đã lỡ mua nhà đất viết giấy tờ tay như trên, giờ phải làm sao để hạn chế rủi ro?Luật sư tư vấnCăn cứ điều 101 Luật Đất đai năm 2013 và điều 82 Nghị định 43/2014/NĐ-CP (được sửa đổi, bổ sung bởi Nghị định 01/2017/NĐ-CP), trường hợp mua nhà đất bằng giấy tờ tay trước ngày 1/1/2008 mà chưa được cấp Giấy chứng nhận quyền sử dụng đất, quyền sở hữu nhà ở và tài sản khác gắn liền với đất (thực tế thường gọi là sổ đỏ) nếu đảm bảo được các điều kiện sau đây sẽ được cấp sổ đỏ:- Hộ gia đình, cá nhân đang sử dụng đất đã được sử dụng ổn định từ trước ngày 1/7/2004.- Không vi phạm pháp luật về đất đai.- Được UBND cấp xã xác nhận là đất không có tranh 

## Chuyển dữ liệu dạng text về ma trận (n x m) bằng TF-IDF

* Bạn cần viết đoạn mã tương ứng trong cell bên dưới. Theo các bước được gợi ý

In [14]:
# load dữ liệu các stopwords
stop_word=['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh','anh_ấy']
num=0
for i in data_train.data:
    for j in i.split():
        if j in stop_word:
            num+=1
print(f'số lượng stopwords: {num}')
print(stop_word)
# Chuyển hoá dữ liệu text về dạng vector TF
#     - loại bỏ từ dừng
#     - sinh từ điển
count_vectorizer=CountVectorizer(stop_words=stop_word)
count_matrix=count_vectorizer.fit_transform(data_train.data)
# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận
# Input: Dữ liệu 2 chiều dạng numpy.array, mảng nhãn id dạng numpy.array
tfidf_transformer=TfidfTransformer()
data_preprocessed=tfidf_transformer.fit_transform(count_matrix)


X = data_preprocessed # thuoc tinh
Y = data_train.target #nhan

print(f"\nSố lượng từ trong từ điển: {len(count_vectorizer.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {X.shape}")
print(f"Kích thước nhãn tương ứng: {Y.shape}")

số lượng stopwords: 823
['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']

Số lượng từ trong từ điển: 10665
Kích thước dữ liệu sau khi xử lý: (970, 10665)
Kích thước nhãn tương ứng: (970,)


In [20]:
print(X[100].toarray())
print(Y[100])

[[0. 0. 0. ... 0. 0. 0.]]
2


In [16]:
sum(sum(X[100].toarray() != 0))

252

In [17]:
print(X[100])

  (0, 10628)	0.07998012341581921
  (0, 10594)	0.06643423062733328
  (0, 10589)	0.019579751399617266
  (0, 10587)	0.08465196025310555
  (0, 10583)	0.05066574005668484
  (0, 10567)	0.03570114277766295
  (0, 10555)	0.03234845770816448
  (0, 10544)	0.036267506369680284
  (0, 10521)	0.03542866610014448
  (0, 10509)	0.030614599891615173
  (0, 10507)	0.033988434135753066
  (0, 10502)	0.0229266232912506
  (0, 10499)	0.04373751155166477
  (0, 10475)	0.050027910361305654
  (0, 10473)	0.03228434586375393
  (0, 10454)	0.028736356417743473
  (0, 10450)	0.039988091716163014
  (0, 10430)	0.031229164660483782
  (0, 10410)	0.04942280461394976
  (0, 10401)	0.09826301292660627
  (0, 10377)	0.04351517071233015
  (0, 10375)	0.0219105597578166
  (0, 10338)	0.03570114277766295
  (0, 10328)	0.059517076919829444
  (0, 9906)	0.0298046855473653
  :	:
  (0, 2009)	0.029561666264261727
  (0, 1782)	0.10851361197736671
  (0, 1762)	0.030614599891615173
  (0, 1751)	0.02370047283858495
  (0, 1744)	0.07180308297167581
  